# Demo workflows

Below are demonstrated the use of this library as a Python package and from the command line interface. Note that one step demonstrated below is to save datasets you need to run the local examples.

In [1]:
import xarray as xr
import ocean_data_gateway as odg
import cf_xarray
import pandas as pd
import extract_model as em
import ocean_model_skill_assessor as omsa
import numpy as np
from IPython import display
from glob import glob

In [2]:
criteria = {
    "salt": {
        "name": "sea_water_practical_salinity$",
    },
}

var_def = {
    "salt": {"units": "psu", "fail_span": [-10, 60], "suspect_span": [-1, 45]},
}


In [3]:
url = 'https://gist.githubusercontent.com/kthyng/c3cc27de6b4449e1776ce79215d5e732/raw/af448937e4896535e36ef6522df8460e8f928cd6/my_custom_criteria.py'
criteria = odg.return_response(url)
criteria

{'ssh': {'standard_name': 'sea_surface_height$|sea_surface_elevation|sea_surface_height_above_sea_level$',
  'name': '(?i)sea_surface_elevation(?!.*?_qc)|(?i)sea_surface_height_above_sea_level_geoid_mllw$|(?i)zeta$|(?i)Sea Surface Height(?!.*?_qc)|(?i)Water Surface above Datum(?!.*?_qc)'},
 'temp': {'name': '(?i)temp$|(?i)temperature$|(?i)tem$|(?i)s.sea_water_temperature$|(?i)temperature(?!.*(skin|ground|air|_qc))'},
 'salt': {'standard_name': 'sea_water_salinity$|sea_water_practical_salinity$',
  'name': '(?i)salinity(?!.*(soil|_qc))|(?i)sea_water_salinity$|(?i)sea_water_practical_salinity$|(?i)salinity$|(?i)salt$|(?i)sal$|(?i)s.sea_water_practical_salinity$'},
 'u': {'standard_name': 'eastward_sea_water_velocity$|sea_water_x_velocity|surface_eastward_sea_water_velocity',
  'name': '(?i)eastward_sea_water_velocity(?!.*?_qc)|(?i)sea_water_x_velocity(?!.*?_qc)|(?i)uo(?!.*?_qc)'},
 'v': {'standard_name': 'northward_sea_water_velocity$|sea_water_y_velocity|surface_northward_sea_water_velo

In [ ]:
# # for ciofs:
# kw = {
#     "min_lon": -156,
#     "max_lon": -148,
#     "min_lat": 56,
#     "max_lat": 62,
#     "min_time": '2022-5-20',
#     "max_time": '2022-5-23',
# }

In [ ]:
# # setup Data search object
# data = odg.Gateway(kw=kw, approach='region')

In [ ]:
#odg.Gateway?

In [ ]:
#len(data.dataset_ids)

In [ ]:
# valid_ids = []
# valid_dss = []
# for k in range(238, len(data.dataset_ids)): 
#    if data[data.dataset_ids[k]]: 
#        if ('sea_water_salinity' in data[data.dataset_ids[k]].variables) | ('sea_water_practical_salinity' in data[data.dataset_ids[k]].variables):
#            print(k, '  ', data.dataset_ids[k])
#            valid_ids.append(k)
#            valid_dss.append(data.dataset_ids[k])           

In [ ]:
# # for ciofs
# valid_ids = [51, 83, 219, 238]
# valid_dss = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a', 'cdmo_nerrs_3b040240', 'nerrs_kachdwq']

In [ ]:
# # plotting just stations with salinity 
# import matplotlib.pyplot as plt

# plot_lat = []
# plot_lon = []
# plot_text = []
# for valid_ds in valid_dss:
#     plot_lat.append(data[valid_ds].latitude)
#     plot_lon.append(data[valid_ds].longitude)
#     # plot_text.append(str(k))

In [ ]:
# # plotting all stations
# plt.plot(plot_lon,plot_lat,'.')
# plt.show()

# Let's test omsa.run - that's where the problem was, originally (with the netcdf file downloaded)
The problem was originally `KeyError: lat, on ocean_model_skill_assessor/main.py:370`. That's within the omsa.run func. So let's run that func piecewise here.


In [ ]:
### test first lines of omsa.run, just to be sure

In [26]:
# these don't really apply to ciofs, but putting here anyway
if xarray_kwargs is None:
    xarray_kwargs = {}

# Set custom criteria
if criteria:
    if isinstance(criteria, str) and criteria[:4] == "http":
        criteria2 = odg.return_response(criteria)
    cf_xarray.set_options(custom_criteria=criteria)

if var_def:
    if isinstance(var_def, str) and var_def[:4] == "http":
        var_def2 = odg.return_response(var_def)

In [27]:
whos

Variable        Type         Data/Info
--------------------------------------
T               NoneType     None
Z               NoneType     None
axds            NoneType     None
cf_xarray       module       <module 'cf_xarray' from <...>s/cf_xarray/__init__.py'>
compute_stats   function     <function compute_stats at 0x7f88080b6830>
criteria        dict         n=6
da              Dataset      <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
data            Dataset      <xarray.Dataset>\nDimensi<...>ing:           -151.72096
data_var        DataArray    <xarray.DataArray 'sea_wa<...>nits:                1e-3
dataset_ids     list         n=2
df              DataFrame                          obs<...>\n\n[67 rows x 2 columns]
display         module       <module 'IPython.display'<...>ages/IPython/display.py'>
dsm             Dataset      <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
em              module       <module 'extract_model' f<...>tract_model/__init__.py'>
er

### Testing read_model

In [4]:
def read_model(loc_model, xarray_kwargs, time_range=None):
    """Read in model output input by user.

    Parameters
    ----------
    loc_model : str
        Relative or absolute, local or nonlocal path to model output.
    xarray_kwargs : dict, optional
        Keyword arguments to pass into `xr.open_dataset`.
    time_range: list
        [min_time, max_time] for desired time range of search where each
        are strings that can be interpreted with pandas `Timestamp`.

    Returns
    -------
    xarray Dataset containing model output.
    """

    dsm = xr.open_dataset(loc_model, **xarray_kwargs)

    # add more cf-xarray info
    dsm = dsm.cf.guess_coord_axis()

    # drop duplicate time indices if present
    # also limit the time range of the model output to what we are requesting from the data to
    # not waste extra time on the model interpolation
    # https://stackoverflow.com/questions/51058379/drop-duplicate-times-in-xarray
    _, index = np.unique(dsm.cf["T"], return_index=True)

    if time_range:
        dsm = dsm.cf.isel(T=index).cf.sel(T=slice(time_range[0], time_range[1]))

    # force longitude to be from -180 to 180
    lkey = dsm.cf["longitude"].name
    dsm[lkey] = dsm.cf["longitude"].where(
        dsm.cf["longitude"] < 180, dsm.cf["longitude"] - 360
    )

    return dsm

In [5]:
# this was done with the file downloaded from: 
# https://thredds/ncss/NOAA_COOPS_OFS_CIOFS.nc?var=salt&north=61.5247&west=-156.4852&east=-148.9251&south=56.7004&horizStride=1&time_start=2022-05-20T00%3A00%3A00Z&time_end=2022-05-23T18%3A00%3A00Z&timeStride=1&vertCoord=1&vertStride=1&addLatLon=true&accept=netcdf
model_url = '/home/anapaula/Downloads/NOAA_COOPS_OFS_CIOFS.nc'
loc_model = model_url
# xarray_kwargs = xarray_kwargs={'chunks': {'time': 1, 'depth': 1},"decode_coords":"all"}
xarray_kwargs = xarray_kwargs={'chunks': {'time': 1, 'depth': 1}}
time_range=['2022-5-20','2022-5-23']

In [6]:
dsm = read_model(loc_model, xarray_kwargs, time_range)
# dsm = read_model(loc_model, time_range)

In [7]:
dsm
# dsm.dims
# dsm.coords
# dsm.cf
# dsm.lon_rho.values

<xarray.Dataset>
Dimensions:     (ocean_time: 67, s_rho: 1, eta_rho: 1044, xi_rho: 724)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2022-05-21 ... 2022-05-23T18:00:00
  * s_rho       (s_rho) float32 -0.01667
    lat_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
    lon_rho     (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 1, 1044, 724), meta=np.ndarray>
    Cs_r        (s_rho) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    zeta        (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(67, 1044, 724), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1044, 724), meta=np.ndarray>
    hc          float32 ...
Attributes: (12/67)
    file:                          nos.ciofs.fields.forecast.20220621.t18z_00...
    format:                        netCDF-4/HDF5 file
    type:                          ROMS/TOMS history file
    var_info:                      varinfo.dat
    rst_file:                      nos.ciofs.rst.forecast.20220621.t18z.nc
    his_base:                      nos.ciofs.fields.forecast.20220621.t18z
    ...                            ...
    summary:                       The Cook Inlet Ocean Forecast System uses ...
    time_coverage_duration:        PT24H
    time_coverage_end:             2022-06-23T23:00:00
    time_coverage_resolution:      PT1H
    time_coverage_start:           2022-06-23T00:00:00
    History:                       Translated to CF-1.0 Conventions by Netcdf...

Check if bounding box works

In [28]:
def find_bbox(ds):
    """Determine bounds and boundary of model.
    Parameters
    ----------
    ds: Dataset
        xarray Dataset containing model output.
    Returns
    -------
    List containing geographic bounding box of model output: [min_lon, min_lat, max_lon, max_lat] and Nx2 array of boundary of model.
    """

    try:
        lon = ds.cf["longitude"].values
        lat = ds.cf["latitude"].values

    except KeyError as e:
        # In case there are multiple grids, just take first one;
        # they are close enough
        lon = list(ds.cf[["longitude"]].coords.keys())[0].values
        lat = list(ds.cf[["latitude"]].coords.keys())[0].values

    min_lon = lon.min()
    max_lon = lon.max()
    min_lat = lat.min()
    max_lat = lat.max()
    #     min_lon = float(ds[lon].min())
    #     max_lon = float(ds[lon].max())
    #     min_lat = float(ds[lat].min())
    #     max_lat = float(ds[lat].max())
    #     import pdb; pdb.set_trace()
    if lon.ndim == 2:
        lonb = np.concatenate((lon[:, 0], lon[-1, :], lon[::-1, -1], lon[0, ::-1]))
        latb = np.concatenate((lat[:, 0], lat[-1, :], lat[::-1, -1], lat[0, ::-1]))
    elif lon.ndim == 1:
        nlon, nlat = ds["lon"].size, ds["lat"].size
        lonb = np.concatenate(([lon[0]] * nlat, lon[:], [lon[-1]] * nlat, lon[::-1]))
        latb = np.concatenate((lat[:], [lat[-1]] * nlon, lat[::-1], [lat[0]] * nlon))
    boundary = np.vstack((lonb, latb)).T

    return [min_lon, min_lat, max_lon, max_lat], boundary

In [30]:
bbox_model, boundary = find_bbox(dsm)
# if bbox is None:
bbox = bbox_model

In [31]:
bbox

[-156.48529, 56.700493, -148.92513, 61.524776]

### read_model works, good. Now let's test ocean data gateway for stations - that's where the problem was, originally (with the netcdf file downloaded)



In [8]:
# test ocean data gateway

# best datasets - ciofs
dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a']

parallel = False
variables='salt'
readers=[odg.erddap]
local=None
erddap={'known_server': 'ioos'}
axds=None
skip_units=False

kwargs = dict(
        criteria=criteria,
        var_def=var_def,
        approach='stations',
        parallel=parallel,
        variables=variables,
        readers=readers,
        local=local,
        erddap=erddap,
        axds=axds,
        skip_units=skip_units,
    )

time_range=['2022-5-20','2022-5-23']
kw = dict(min_time=time_range[0], max_time=time_range[1])
kwargs["kw"] = kw

kwargs["stations"] = dataset_ids

search = odg.Gateway(**kwargs)

In [9]:
# check what's available
search.kwargs
# search.kwargs_all
# search.criteria
# search.var_def
# search.sources
# search.dataset_ids

{'criteria': {'ssh': {'standard_name': 'sea_surface_height$|sea_surface_elevation|sea_surface_height_above_sea_level$',
   'name': '(?i)sea_surface_elevation(?!.*?_qc)|(?i)sea_surface_height_above_sea_level_geoid_mllw$|(?i)zeta$|(?i)Sea Surface Height(?!.*?_qc)|(?i)Water Surface above Datum(?!.*?_qc)'},
  'temp': {'name': '(?i)temp$|(?i)temperature$|(?i)tem$|(?i)s.sea_water_temperature$|(?i)temperature(?!.*(skin|ground|air|_qc))'},
  'salt': {'standard_name': 'sea_water_salinity$|sea_water_practical_salinity$',
   'name': '(?i)salinity(?!.*(soil|_qc))|(?i)sea_water_salinity$|(?i)sea_water_practical_salinity$|(?i)salinity$|(?i)salt$|(?i)sal$|(?i)s.sea_water_practical_salinity$'},
  'u': {'standard_name': 'eastward_sea_water_velocity$|sea_water_x_velocity|surface_eastward_sea_water_velocity',
   'name': '(?i)eastward_sea_water_velocity(?!.*?_qc)|(?i)sea_water_x_velocity(?!.*?_qc)|(?i)uo(?!.*?_qc)'},
  'v': {'standard_name': 'northward_sea_water_velocity$|sea_water_y_velocity|surface_nort

Now checking if the datasets plot works. It **should**

In [33]:
def prep_plot(search):
    """Put together inputs for map plot."""

    sub = search.meta.loc[
        search.dataset_ids,
        [
            "geospatial_lon_min",
            "geospatial_lat_min",
            "geospatial_lon_max",
            "geospatial_lat_max",
        ],
    ]
    lls = sub.values
    names = list(sub.index.values)

    # put out stationary data
    istations = lls[:, 0] == lls[:, 2]

    # temporarily remove dataset_ids that aren't a station
    ids_to_remove = list(np.array(names)[~istations])
    for id_remove in ids_to_remove:
        ind = search.sources[0].dataset_ids.index(id_remove)
        search.sources[0].dataset_ids.pop(ind)

    sub = search.meta.loc[
        search.dataset_ids,
        [
            "geospatial_lon_min",
            "geospatial_lat_min",
            "geospatial_lon_max",
            "geospatial_lat_max",
        ],
    ]
    lls = sub.values
    names = list(sub.index.values)

    # put out stationary data
    istations = lls[:, 0] == lls[:, 2]
    lls_stations = lls[istations, :2]
    names_stations = list(np.array(names)[istations])
    if len(names_stations) == 0:
        names_stations = None
        lls_stations = None

    # pull out data over range
    lls_box = lls[~istations]
    names_boxes = list(np.array(names)[~istations])

    if len(names_boxes) == 0:
        names_boxes = None
        lls_box = None

    return lls_stations, names_stations, lls_box, names_boxes


In [ ]:
lls_stations, names_stations, lls_box, names_boxes = prep_plot(search)

In [37]:
names_boxes

In [ ]:
figname_map = 'figure1'
omsa.map.plot(
    lls_stations=lls_stations,
    names_stations=names_stations,
    lls_boxes=lls_box,
    names_boxes=names_boxes,
    boundary=boundary,
    res="10m",
    figname=figname_map,
)

### Ocean data gateway works, good. Now let's make sure prep extract model works

In [10]:
def prep_em(input_data):
    """Prepare to run extract_model."""

    if isinstance(input_data, pd.DataFrame):
        data = input_data
        tname = data.cf["T"].name
        data[tname] = pd.to_datetime(data.cf["T"])
        data = data.set_index(data.cf["T"])
    else:
        data = input_data
    lon = float(data.cf["longitude"].values[0])
    lat = float(data.cf["latitude"].values[0])
    T = None
    # only compare surface
    Z = None

    return data, lon, lat, T, Z

In [11]:
data, lon, lat, T, Z = prep_em(search[dataset_ids[0]])

In [12]:
data

<xarray.Dataset>
Dimensions:                                                                (
                                                                            timeseries: 1,
                                                                            time: 384)
Coordinates:
    latitude                                                               (timeseries) float64 ...
    longitude                                                              (timeseries) float64 ...
  * time                                                                   (time) datetime64[ns] ...
Dimensions without coordinates: timeseries
Data variables: (12/30)
    station                                                                (timeseries) object ...
    rowSize                                                                (timeseries) int32 ...
    z                                                                      (time, timeseries) float64 ...
    mass_concentration_of_chlorophyll_in_sea_water                         (time, timeseries) float64 ...
    mass_concentration_of_chlorophyll_in_sea_water_qc_agg                  (time, timeseries) float64 ...
    mass_concentration_of_chlorophyll_in_sea_water_qc_tests                (time, timeseries) int32 ...
    ...                                                                     ...
    sea_surface_height_above_sea_level_geoid_local_station_datum           (time, timeseries) float64 ...
    sea_surface_height_above_sea_level_geoid_local_station_datum_qc_agg    (time, timeseries) float64 ...
    sea_surface_height_above_sea_level_geoid_local_station_datum_qc_tests  (time, timeseries) float64 ...
    sea_water_ph_reported_on_total_scale                                   (time, timeseries) float64 ...
    sea_water_ph_reported_on_total_scale_qc_agg                            (time, timeseries) float64 ...
    sea_water_ph_reported_on_total_scale_qc_tests                          (time, timeseries) float64 ...
Attributes: (12/54)
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             feedback@axiomdatascience.com
    contributor_name:              Axiom Data Science
    contributor_role:              processor
    contributor_role_vocabulary:   NERC
    ...                            ...
    station_id:                    20698
    summary:                       Timeseries data from 'Seldovia Deep Water ...
    time_coverage_end:             2022-06-22T23:45:00Z
    time_coverage_start:           2004-01-01T09:00:00Z
    title:                         Seldovia Deep Water Quality
    Westernmost_Easting:           -151.72096

In [ ]:
lon

In [ ]:
data

# great, that works too. Now let's check the kwargs for em.select   
            kwargs = dict(
                da=dsm.cf[variable].cf.isel(Z=0).cf.sel(lon=slice(lon - 5, lon + 5), lat=slice(lat - 5, lat + 5)),
                longitude=lon,
                latitude=lat,
                T=T,
                iZ=Z,
                locstream=True,
            )   
            
The main issue here is with subsetting the model. Ciofs dataset has as dimensions ocean_time, s_rho, eta_rho and xi_rho. And the coordinates are ocean_time, s_rho, lat_rho, and lon_rho.
So we can't subset directly on lat_rho and lon_rho.

In [ ]:
dsm.salt.cf.sel(longitude=slice(lon - 5, lon + 5), latitude=slice(lat - 5, lat + 5))

In [ ]:
# KeyError: 'no index found for coordinate lat_rho'

In [ ]:
dsm.salt.cf.sel(lon=slice(lon - 5, lon + 5), lat=slice(lat - 5, lat + 5))

In [ ]:
# KeyError: 'lat is not a valid dimension or coordinate'

## Ok, let's work on those kwargs in a different way

In [14]:
# careful with the square brackets here. If you don't put it, it raises an error
variable=['salt']        
# non-cf stuff
# da = dsm.where((dsm.lon_rho > lon-2) & (dsm.lon_rho < lon+2) & (dsm.lat_rho > lat-2) & (dsm.lat_rho < lat+2), drop=True).cf.isel(Z=0)

# cf'd
da = dsm.where((dsm.cf['longitude'] > lon-2) & 
               (dsm.cf['longitude'] < lon+2) & 
               (dsm.cf['latitude'] > lat-2) & 
               (dsm.cf['latitude'] < lat+2), drop=True).cf.isel(Z=0)

kwargs = dict(da=da.cf[variable],
              longitude=lon,
              latitude=lat,
              T=T,
              iZ=Z,
              locstream=True,
              )
model_var = em.select(**kwargs) # xarray.dataset
# sel_array= sel_dataset.to_array() # xarray.dataarray
# model_var = sel_array.to_dataset()# xarray.dataset


if isinstance(model_var, xr.Dataset):
    print('opa')
else:
    pass # put something from the code
    



opa


In [16]:
model_var.salt.values

array([[32.11937 ],
       [32.120323],
       [32.12048 ],
       [32.12042 ],
       [32.120407],
       [32.121483],
       [32.122093],
       [32.12189 ],
       [32.121944],
       [32.12214 ],
       [32.12236 ],
       [32.12201 ],
       [32.12133 ],
       [32.12085 ],
       [32.12066 ],
       [32.120144],
       [32.12055 ],
       [32.121044],
       [32.12155 ],
       [32.121906],
       [32.122063],
       [32.12228 ],
       [32.122494],
       [32.1226  ],
       [32.122013],
       [32.12153 ],
       [32.12096 ],
       [32.119614],
       [32.120136],
       [32.12145 ],
       [32.121788],
       [32.122234],
       [32.122593],
       [32.122734],
       [32.122784],
       [32.122845],
       [32.122864],
       [32.122925],
       [32.12264 ],
       [32.12235 ],
       [32.12191 ],
       [32.121918],
       [32.122208],
       [32.122433],
       [32.12244 ],
       [32.1225  ],
       [32.122513],
       [32.122505],
       [32.12261 ],
       [32.12278 ],


In [17]:
whos

Variable        Type        Data/Info
-------------------------------------
T               NoneType    None
Z               NoneType    None
axds            NoneType    None
cf_xarray       module      <module 'cf_xarray' from <...>s/cf_xarray/__init__.py'>
criteria        dict        n=6
da              Dataset     <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
data            Dataset     <xarray.Dataset>\nDimensi<...>ing:           -151.72096
dataset_ids     list        n=2
display         module      <module 'IPython.display'<...>ages/IPython/display.py'>
dsm             Dataset     <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
em              module      <module 'extract_model' f<...>tract_model/__init__.py'>
erddap          dict        n=1
glob            function    <function glob at 0x7f888166ce50>
kw              dict        n=2
kwargs          dict        n=6
lat             float       59.44099
loc_model       str         /home/anapaula/Downloads/NOAA_COOP

In [22]:
# careful with the square brackets here
data_var = data.cf['salt']
model_var = model_var.cf['salt']

In [23]:
whos

Variable        Type         Data/Info
--------------------------------------
T               NoneType     None
Z               NoneType     None
axds            NoneType     None
cf_xarray       module       <module 'cf_xarray' from <...>s/cf_xarray/__init__.py'>
criteria        dict         n=6
da              Dataset      <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
data            Dataset      <xarray.Dataset>\nDimensi<...>ing:           -151.72096
data_var        DataArray    <xarray.DataArray 'sea_wa<...>nits:                1e-3
dataset_ids     list         n=2
display         module       <module 'IPython.display'<...>ages/IPython/display.py'>
dsm             Dataset      <xarray.Dataset>\nDimensi<...> Conventions by Netcdf...
em              module       <module 'extract_model' f<...>tract_model/__init__.py'>
erddap          dict         n=1
glob            function     <function glob at 0x7f888166ce50>
kw              dict         n=2
kwargs          dict         n

In [24]:
from ocean_model_skill_assessor import compute_stats
df = omsa.stats._align(data_var, model_var)

# Ok, for now this works. Let's tackle the rest tomorrow!

In [ ]:
# might need this later
# dsm.rename_dims({"s_rho":"z","eta_rho":"lon_rho","xi_rho":"lat_rho"})
# dsm.swap_dims({"s_rho":"z","eta_rho":"lon_rho","xi_rho":"lat_rho"})

## Python library

This package can be used as a Python package or from the command line. Here we demonstrate its use as a Python package.

### Nonlocal reader: stations

This demonstrates the case in which your data files for comparing with model output are not available locally, but you know their names and where to find them (in this case, at the IOOS ERDDAP server). More information about these inputs is available in the [API docs](https://ocean-model-skill-assessor.readthedocs.io/en/latest/api.html).

In [ ]:
var_def

In [ ]:
criteria

In [ ]:
from ocean_data_gateway import read_model

In [ ]:
# FOR CIOFS:

# model_url = 'http://thredds.aoos.org/thredds/dodsC/NOAA_COOPS_OFS_CIOFS.nc'
model_url = 'NOAA_COOPS_OFS_CIOFS.nc'


# best datasets - ciofs
dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a'] 
    
search = omsa.run(loc_model = model_url,
                  approach='stations',
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-5-20','2022-5-23'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  stations = dataset_ids,
                  figname_map='nonlocal_library.png',
                  figname_data_prefix='nonlocal_library_',
                  parallel=False
)

#### Save local data files

Here we save the data files to use for the local examples.

In [40]:
# CIOFS dataset_ids = ['nerrs_kacsdwq', 'cdmo_nerrs_3b00077a'] 
search['nerrs_kacsdwq'].squeeze().to_dataframe().to_csv('nerrs_kacsdwq')
search['cdmo_nerrs_3b00077a'].to_netcdf('cdmo_nerrs_3b00077a.nc')

### Nonlocal reader: region

This demonstrates the case in which your data files for comparing with model output are not available locally, and you want to perform a search in time and space. By default this would search in the spatial bounding box of the model output, but here we instead input a smaller bounding box so as to limit the number of datasets found and used. For several of the datasets, the model output isn't available (must be determined to be on land). 

In [ ]:
omsa.set_criteria(criteria)

bbox = [-153, 57, -145, 65]
search = omsa.run(
                  loc_model=model_url,
                  approach='region',
                  bbox=bbox,
                  criteria=criteria,
                  var_def=var_def,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-5-20','2022-5-23'], 
                  variables=['salt'],
                  readers=[odg.erddap],
                  erddap={
                      'known_server': 'ioos'
                  },
                  figname_map='nonlocal_library_region.png',
                  figname_data_prefix='nonlocal_library_region_'
)

### Local reader

This demonstrates the case in which your data files for comparing with model output are available locally.

In [ ]:
omsa.set_criteria(criteria)

filenames = [
             'nerrs_kacsdwq.csv',
             'cdmo_nerrs_3b00077a.nc',
            ]

skip_units = True

search = omsa.run(
                  loc_model='NOAA_COOPS_OFS_CIOFS.nc',
                  approach='region',
                  criteria=criteria,
                  var_def=var_def,
                  skip_units=skip_units,
                  xarray_kwargs={'chunks': {'time': 1, 'depth': 1}},
                  time_range=['2022-5-20','2022-5-23'],  
                  variables=['salt'],
                  readers=[odg.local],
                  local={'filenames': filenames},
                  figname_map='local_library.png',
                  figname_data_prefix='local_library_'
)

## Command line interface

Here we demonstrate the use of the command line interface mode of the package. The config yaml file must be modified for the necessary inputs. These are the same examples as above, but accomplished via the command line interface instead of the Python package.

### Local reader

In [ ]:
config_file = 'config_local.yaml'

In [ ]:
# %load config_local.yaml
---
approach: "stations"
loc_model: "https://thredds.cencoos.org/thredds/dodsC/CENCOOS_CA_ROMS_FCST.nc"
axds:
bbox:
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
figname_map: "local_cli.png"
figname_data_prefix: "local_cli_"
local:
  filenames:
    - "nerrs_kacsdwq.csv"
    - "cdmo_nerrs_3b00077a.nc"
only_search: false
only_searchplot: false
parallel: true
readers:
  - "local"
run_qc: false
skip_units: true
stations:
time_range:
  - "2022-5-20"
  - "2022-5-23"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1


In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("local_cli.png")

In [ ]:
display.Image('local_cli_nerrs_kacsdwq.csv_salt.png')

In [ ]:
display.Image('local_cli_cdmo_nerrs_3b00077a.nc_salt.png')

### Nonlocal workflow: stations

In [ ]:
config_file = 'config_nonlocal.yaml'

In [ ]:
# %load config_nonlocal.yaml
---
approach: "stations"
loc_model: "http://thredds.aoos.org/thredds/dodsC/NOAA_COOPS_OFS_CIOFS.nc"
axds:
bbox:
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
  known_server:
    "ioos"
figname_map: "nonlocal_cli.png"
figname_data_prefix: "nonlocal_cli_"
local:
only_search: false
only_searchplot: false
parallel: true
readers:
  - "erddap"
run_qc: false
skip_units: true
stations:
  - "nerrs_kacsdwq"
  - "cdmo_nerrs_3b00077a"
time_range:
  - "2022-5-20"
  - "2022-5-23"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1


In [ ]:
whos

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli.png")

In [ ]:
display.Image('nonlocal_cli_edu_humboldt_tdp_salt.png')

In [ ]:
display.Image('nonlocal_cli_bodega-bay-bml_wts_salt.png')

### Nonlocal reader: region

In [ ]:
config_file = 'config_nonlocal_region.yaml'

In [ ]:
# %load config_nonlocal_region.yaml
---
approach: "region"
loc_model: "http://thredds.aoos.org/thredds/dodsC/NOAA_COOPS_OFS_CIOFS.nc"
axds:
bbox:
  - -124.5
  - 40
  - -123.5
  - 42
criteria:
  salt:
    name:
      "sea_water_practical_salinity$"
erddap:
  known_server:
    "ioos"
figname_map: "nonlocal_cli_region.png"
figname_data_prefix: "nonlocal_cli__region"
local:
only_search: false
only_searchplot: false
parallel: true
readers:
  - "erddap"
run_qc: false
skip_units: true
stations:
time_range:
  - "2021-9-1"
  - "2021-9-8"
var_def:
  salt:
    units: "psu"
    fail_span:
      - -10
      - 60
    suspect_span:
      - -1
      - 45
variables:
  - "salt"
xarray_kwargs:
  chunks:
    time:
      1
    depth:
      1

In [ ]:
!python ../ocean_model_skill_assessor/CLI.py $config_file

In [ ]:
display.Image("nonlocal_cli_region.png")

In [ ]:
display.Image('nonlocal_cli__regionexploratorium-pco2-buoy_salt.png')

In [ ]:
display.Image('nonlocal_cli__regionedu_calpoly_marine_morro_salt.png')

In [ ]:
display.Image('nonlocal_cli__regionsccoos_ucsd_salt.png')